In [1]:
!pip install peft accelerate datasets bitsandbytes
!pip install -U transformers accelerate
!pip install -U qwen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.7/522.7 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.5.3
    Uninstalling joblib-1.5.3:
      Successfully uninstalled joblib-1.5.3
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_name = "Qwen/Qwen3-14B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
)
model.gradient_checkpointing_enable()
model.config.use_cache = False


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00005-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [2]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["q_proj",  "k_proj"],
    lora_dropout = 0.1,
    bias = "none",
    task_type = "CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 20,971,520 || all params: 14,789,278,720 || trainable%: 0.1418


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from pathlib import Path
TRAIN_DIR = Path("/content/drive/MyDrive/dataset/train.jsonl")

In [5]:
import json
from datasets import Dataset

def load_and_format(json_path):
  dataset = []
  with open(json_path, 'r') as f:
    for line in f:
      data = json.loads(line.strip())
      histories = data["input"]["history"]
      que = data["input"]["current"]
      refs = data["input"]["reference"]

      if histories:
        limited = histories[-2:]
        history_text = "\n".join(f"{h[0]}: {h[1]}" for h in limited)
      else:
        history_text = ""

      ref_sorted = sorted(refs, key=lambda x: x["score"], reverse=True)
      ref_text = "\n".join(f"[{i+1}] {r['text']}" for i, r in enumerate(ref_sorted))
      system = (
          "Instruction: \n"
          "You are a RAG answer generator."
          "Use the reference documents and history conversation as the main source of information to answer the question."
          "If the reference contains partial clues, synthesize them to answer the question."
          "Answer concisely (1–2 sentences)."
          "If the reference is empty, answer: \"I don't know.\"\n"
      )
      user = (
          f"Reference:\n{ref_text}\n\n"
          f"History Coversation: {history_text}\n\n"
          f"Current Question: {que}\n\n"
          "Answer:\n"
      )
      dataset.append({"messages": [
          {"role": "system", "content": system},
          {"role": "user", "content": user},
          {"role": "assistant", "content": data["output"]}
      ]})
    return Dataset.from_list(dataset)


In [6]:
example = load_and_format(TRAIN_DIR)[121]
print(example)
full_ids = tokenizer.apply_chat_template(
    example["messages"],
    tokenize=True,
    add_generation_prompt=False,
    return_tensors="pt"
)

num_tokens = full_ids.shape[1]
print("Total tokens:", num_tokens)

{'messages': [{'content': 'Instruction: \nYou are a RAG answer generator.Use the reference documents and history coversations as the main source of information to answer the question.If the reference contains partial clues, synthesize them to answer the question.Answer concisely (1–2 sentences).If the reference is empty, answer: "I don\'t know."\n', 'role': 'system'}, {'content': 'Reference:\n[1] \n"A couple options that I know of: Interactive Brokers offers a ""paper trading"" mode to its account holders that allows you to start with a pretend stack of money and place simulated trades to test trading ideas. They also provide an API that allows you to interface with their platform programmatically for retrieving quotes, placing orders, and the such. As you noted, however, it\'s not free; you must hold a funded brokerage account in order to qualify for access to their platform. In order to maintain an account, there are minimums for required equity and monthly activity (measured in doll

In [7]:
def tokenize(example, max_length=4096):
  full_text = tokenizer.apply_chat_template(
      example["messages"],
      tokenize=False,
      add_generation_prompt=False,
      enable_thinking=False,
      )
  prefix_text = tokenizer.apply_chat_template(
      example["messages"][:-1],
      tokenize=False,
      add_generation_prompt=True,
      enable_thinking=False,
  )
  full_ids = tokenizer(full_text, truncation=True, max_length=max_length, padding = "max_length" ,return_tensors="pt")
  input_ids = full_ids["input_ids"][0]
  prefix = tokenizer(prefix_text, truncation=True, max_length=max_length, return_tensors="pt")
  prefix_len = prefix["input_ids"].shape[1]
  labels = input_ids.clone()
  labels[:prefix_len] = -100
  return {"input_ids": input_ids, "labels": labels}
dataset = load_and_format(TRAIN_DIR)
train_data = dataset.map(tokenize)

Map:   0%|          | 0/673 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

args = TrainingArguments(
    output_dir = "rag_lora",
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    learning_rate =5e-5,
    num_train_epochs=2,
    fp16=True,
    logging_steps=20,
    warmup_ratio=0.05,
    lr_scheduler_type="cosine",
    save_strategy="no",
    report_to="none",
)

trainer = Trainer(
    model = model,
    args = args,
    train_dataset=train_data,
)
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.


Step,Training Loss
20,7.111400


In [ ]:
model.save_pretrained("rag_lora_adapter_qwen3_14B_try3")

In [ ]:
!cp -r rag_lora_adapter_qwen3_14B /content/drive/MyDrive/

cp: cannot stat 'rag_lora_adapter_chatmode': No such file or directory
